# ***Distribución BAÑOS***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_")  and "puebla" in nombre and 'df_alfa_q_puebla' not in nombre and 'financiamientos' not in nombre and 'grupos' not in nombre and 'salarios' not in nombre and 'publicacion' not in nombre and 'df_feb_2024_puebla' not in nombre    #if nombre.startswith('df_') and 'alfa' in nombre and'puebla' in nombre and 'q' not in nombre
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

ImportError: DLL load failed while importing _multiarray_umath: No se puede encontrar el módulo especificado.

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_febrero_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_septiembre_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [3]:
df_alfa_agosto_2024_puebla.columns

Index(['id', 'Categoria', 'Precio', 'propiedad', 'metros_total',
       'metros_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_de_publicacion', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Seguridad',
       'Colonia', 'CP', 'url'],
      dtype='object')

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Categoria':'categoria','Banos_Total':'baño_total','id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido','Precio':'precio',}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','baño_total','precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

## Total de baños

In [4]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

In [5]:
for df in dataframes_list:
    print (df['baño_total'].unique())
    df = df.drop(columns=['categoria'])
    # Asignar la categoría real a cada registro
    df['categoria'] = df['precio'].apply(asignar_categoria)

[ 1.   2.   1.5  3.5  4.   2.5  3.   4.5 31.5 21.   5.5  5.   6.  10.
  9.   7.   8.   6.5  7.5 70.  13.  12.   8.5 17.  12.5]
[ 1  2  3  5  4 21  6  7  9 10  8 13 70]
[ 1.   2.   1.5  3.5  4.   2.5 11.   3.   4.5 31.5 21.   5.5  5.   6.
 10.   9.   7.   8.   6.5  7.5 70.  13.  12.   8.5 17.  12.5]
[ 2.   1.   3.  11.   4.  21.   5.   1.5  2.5  6.   7.   9.  10.   3.5
 31.   8.  70.  13.  41.   5.5 17.   7.5]
[1.  1.5 2.  4.  2.5 3.5 3.  4.5 5.  6.  5.5 7.  8.  6.5 7.5]
[ 3  4  6  5  1  0  2  9  8  7 70 10 15]
[1.  2.  5.  3.  4.  2.5 6.  8.  7. ]
[ 1  2  3  5  4 21  7  6  9 10 31  8 13 70 17]
[ 2.   1.   3.   4.   1.5  5.   0.   6.   7.   9.   8.  10.   4.5 20.
 16. ]


In [6]:
import numpy as np
for df in dataframes_list:    
    condition = (df['baño_total'] % 1 == 0.5) # Valores que contienen .5    
    # Convertir al número entero anterior
    df.loc[condition, 'baño_total'] = np.floor(df['baño_total'])    
    print(df['baño_total'].unique())

[ 1.  2.  3.  4. 31. 21.  5.  6. 10.  9.  7.  8. 70. 13. 12. 17.]
[ 1  2  3  5  4 21  6  7  9 10  8 13 70]
[ 1.  2.  3.  4. 11. 31. 21.  5.  6. 10.  9.  7.  8. 70. 13. 12. 17.]
[ 2.  1.  3. 11.  4. 21.  5.  6.  7.  9. 10. 31.  8. 70. 13. 41. 17.]
[1. 2. 4. 3. 5. 6. 7. 8.]
[ 3  4  6  5  1  0  2  9  8  7 70 10 15]
[1. 2. 5. 3. 4. 6. 8. 7.]
[ 1  2  3  5  4 21  7  6  9 10 31  8 13 70 17]
[ 2.  1.  3.  4.  5.  0.  6.  7.  9.  8. 10. 20. 16.]


In [7]:
def procesar_dataframes(dataframes_list, nombres_df_filtrados):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df.loc[:, 'segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)

        df.loc[:,'baño_total'] = df['baño_total'].clip(lower=1,upper=8) # Limitar los valores de 'baño_total' a un máximo de 8
        #df['baño_total'] = df['baño_total'].clip(upper=8) 
        #df['baño_total'] = pd.cut(df['baño_total'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {nombres_df_filtrados[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, nombres_df_filtrados)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16700\2851665368.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16700\2851665368.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16700\2851665368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

Nombre del DataFrame: df_alfa_agosto_2024_puebla
baño_total  1.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0
segmento                                          
A            10  125   95   29    9    1    0    1
B            26  134   71   25    3    0    0    0
C            57  222   70    9    1    1    0    0
D           146  195   53   13    5    0    0    0
E           294  474  216   70   10    4    3    7
L             0   32   44   41   26    8    2    3
S            14  164  261  183   55   12    7    4
Nombre del DataFrame: df_alfa_febrero_2024_puebla
baño_total    1    2    3   4   5   6  7  8
segmento                                   
A           110  137   63  31   7   0  0  3
B           113  201   78  12   1   0  1  0
C           182  221   53   6   1   1  0  0
D           263  223   26   3   0   0  0  1
E           511  224  125  17   3   4  2  0
L            22   22   17  21   9   4  1  3
S           224  159  153  79  36  12  3  4
Nombre del DataFrame: df_alfa_julio_2024_puebl

In [8]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
baño_total     1.0     2.0     3.0     4.0    5.0    6.0   7.0   8.0
segmento                                                            
A            758.0  1415.0   900.0   377.0   95.0   14.0   8.0  11.0
B            805.0  1780.0   883.0   295.0   59.0   10.0   6.0   0.0
C           1201.0  2317.0   676.0   107.0   18.0    7.0   0.0   2.0
D           1924.0  1906.0   332.0    58.0   21.0    1.0   0.0   3.0
E           3289.0  2012.0  1020.0   300.0   56.0   24.0  16.0  28.0
L            215.0   210.0   250.0   276.0  139.0   65.0  26.0  22.0
S           1401.0  1533.0  1831.0  1309.0  539.0  158.0  50.0  39.0


In [9]:
import os
import plotly.graph_objects as go
import plotly.io as pio

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9']
orden_segmentos=["E", "D", "C", "B","A", "S", "L", ]
def sumar_tablas(dataframes_list):
    tabla_suma = None
    for df in dataframes_list:
        # Aplica el orden a la columna 'segmento'
        df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
        
        tabla_actual = df.groupby(['segmento', 'baño_total']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    guardar_grafico_como_html(fig, 'g_bar_banos', carpeta='assets/graficas') # Para guardar la gráfica como archivo HTML
    return fig

def graficar_tabla(tabla):
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Número de baños",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Segmento",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )    
    
    fig.show()    
    return fig

# Guardar la gráfica como archivo HTML en la carpeta especificada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_16700\2856565146.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16700\2856565146.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = pd.Categorical(df['segmento'], categories=orden_segmentos, ordered=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_16700\2856565146.py:11: SettingWithCopyWarning: 
A value is 

Tabla general de frecuencia:
baño_total   1.0   2.0   3.0   4.0  5.0  6.0  7.0  8.0
segmento                                              
E           3289  2012  1020   300   56   24   16   28
D           1924  1906   332    58   21    1    0    3
C           1201  2317   676   107   18    7    0    2
B            805  1780   883   295   59   10    6    0
A            758  1415   900   377   95   14    8   11
S           1401  1533  1831  1309  539  158   50   39
L            215   210   250   276  139   65   26   22


In [10]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby('baño_total').size()
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    return tabla_suma

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
tabla_suma = sumar_tablas(dataframes_list)
print("Tabla general total de registros:")
print(tabla_suma)


Tabla general total de registros:
baño_total
1.0     9993
2.0    11820
3.0     6199
4.0     2852
5.0      967
6.0      287
7.0      111
8.0      111
dtype: int64


In [11]:
# Calcular el total de registros
total_registros = tabla_suma.sum()
# Calcular el porcentaje que representa cada número de baños único del total
porcentajes = (tabla_suma / total_registros) * 100

# Imprimir los porcentajes
print("Porcentaje de cada número de baños único en relación con el total de registros:", total_registros)
print(porcentajes)


Porcentaje de cada número de baños único en relación con el total de registros: 32340
baño_total
1.0    30.899814
2.0    36.549165
3.0    19.168213
4.0     8.818800
5.0     2.990105
6.0     0.887446
7.0     0.343228
8.0     0.343228
dtype: float64


In [12]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Lista de colores
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']

# Función para sumar tablas
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby('baño_total').size()
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    return tabla_suma

# Función para calcular porcentajes
def calcular_porcentajes(tabla):
    # Calcular los porcentajes
    tabla_porcentajes = tabla / tabla.sum() * 100
    return tabla_porcentajes

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
tabla_suma = sumar_tablas(dataframes_list)
porcentajes = calcular_porcentajes(tabla_suma) # Calcular porcentajes

# Crear el gráfico de pastel con colores especificados
figURE = go.Figure(data=[go.Pie(labels=porcentajes.index, values=porcentajes, sort=False, marker=dict(colors=colores))])
figURE.update_layout( title_x=0.5, title_font_size=20,margin=dict(l=10, r=10, t=10, b=10),  )
figURE.update_traces(rotation=45) # Ángulo de inicio de las porciones del gráfico

# Exportar gráfica como archivo HTML y mostrar la figura
def guardar_grafico_como_html(figURE, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(figURE, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(figURE, 'g_pie_banos', carpeta='assets/graficas')

# Mostrar la figura
figURE.show()